In [29]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 2: Red-Nosed Reports ---</h2><p>Fortunately, the first location The Historians want to search isn't a long walk from the Chief Historian's office.</p>
<p>While the <a href="/2015/day/19">Red-Nosed Reindeer nuclear fusion/fission plant</a> appears to contain no sign of the Chief Historian, the engineers there run up to you as soon as they see you. Apparently, they <em>still</em> talk about the time Rudolph was saved through molecular synthesis from a single electron.</p>
<p>They're quick to add that - since you're already here - they'd really appreciate your help analyzing some unusual data from the Red-Nosed reactor. You turn to check if The Historians are waiting for you, but they seem to have already divided into groups that are currently searching every corner of the facility. You offer to help with the unusual data.</p>
<p>The unusual data (your puzzle input) consists of many <em>reports</em>, one report per line. Each report is a list of numbers called <em>levels</em> that are separated by spaces. For example:</p>
<pre><code>7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9
</code></pre>
<p>This example data contains six reports each containing five levels.</p>
<p>The engineers are trying to figure out which reports are <em>safe</em>. The Red-Nosed reactor safety systems can only tolerate levels that are either gradually increasing or gradually decreasing. So, a report only counts as safe if both of the following are true:</p>
<ul>
<li>The levels are either <em>all increasing</em> or <em>all decreasing</em>.</li>
<li>Any two adjacent levels differ by <em>at least one</em> and <em>at most three</em>.</li>
</ul>
<p>In the example above, the reports can be found safe or unsafe by checking those rules:</p>
<ul>
<li><code>7 6 4 2 1</code>: <em>Safe</em> because the levels are all decreasing by 1 or 2.</li>
<li><code>1 2 7 8 9</code>: <em>Unsafe</em> because <code>2 7</code> is an increase of 5.</li>
<li><code>9 7 6 2 1</code>: <em>Unsafe</em> because <code>6 2</code> is a decrease of 4.</li>
<li><code>1 3 2 4 5</code>: <em>Unsafe</em> because <code>1 3</code> is increasing but <code>3 2</code> is decreasing.</li>
<li><code>8 6 4 4 1</code>: <em>Unsafe</em> because <code>4 4</code> is neither an increase or a decrease.</li>
<li><code>1 3 6 7 9</code>: <em>Safe</em> because the levels are all increasing by 1, 2, or 3.</li>
</ul>
<p>So, in this example, <code><em>2</em></code> reports are <em>safe</em>.</p>
<p>Analyze the unusual data from the engineers. <em>How many reports are safe?</em></p>
</article>


In [30]:
tests = [
    {
        "name": "Example",
        "s": """
                7 6 4 2 1
                1 2 7 8 9
                9 7 6 2 1
                1 3 2 4 5
                8 6 4 4 1
                1 3 6 7 9
        """,
        "expected": 2,
    },
]


class Reports(Str):
    def __init__(self, s: str) -> None:
        self.reports = [
            [int(r) for r in re.findall(r"\d+", l.strip())]
            for l in s.strip().splitlines()
        ]

    def count_safe_reports(self) -> int:
        return sum(1 for r in self.reports if self.safe(r))

    @classmethod
    def safe(cls, report: list[int]) -> bool:
        positive = report[0] < report[-1]

        return all(
            (report[i - 1] < report[i]) == positive
            and 1 <= abs(report[i - 1] - report[i]) <= 3
            for i in range(1, len(report))
        )


@test(tests=tests)
def partI_test(s: str) -> int:
    return Reports(s).count_safe_reports()


Test Example passed, for partI_test.
Success


In [31]:
with open("../input/day2.txt") as f:
    puzzle = f.read()

print(f"Part I: {Reports(puzzle).count_safe_reports()}")

Part I: 269


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>269</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The engineers are surprised by the low number of safe reports until they realize they forgot to tell you about the <span title="I need to get one of these!">Problem Dampener</span>.</p>
<p>The Problem Dampener is a reactor-mounted module that lets the reactor safety systems <em>tolerate a single bad level</em> in what would otherwise be a safe report. It's like the bad level never happened!</p>
<p>Now, the same rules apply as before, except if removing a single level from an unsafe report would make it safe, the report instead counts as safe.</p>
<p>More of the above example's reports are now safe:</p>
<ul>
<li><code>7 6 4 2 1</code>: <em>Safe</em> without removing any level.</li>
<li><code>1 2 7 8 9</code>: <em>Unsafe</em> regardless of which level is removed.</li>
<li><code>9 7 6 2 1</code>: <em>Unsafe</em> regardless of which level is removed.</li>
<li><code>1 <em>3</em> 2 4 5</code>: <em>Safe</em> by removing the second level, <code>3</code>.</li>
<li><code>8 6 <em>4</em> 4 1</code>: <em>Safe</em> by removing the third level, <code>4</code>.</li>
<li><code>1 3 6 7 9</code>: <em>Safe</em> without removing any level.</li>
</ul>
<p>Thanks to the Problem Dampener, <code><em>4</em></code> reports are actually <em>safe</em>!</p>
<p>Update your analysis by handling situations where the Problem Dampener can remove a single level from unsafe reports. <em>How many reports are now safe?</em></p>
</article>


In [32]:
from typing import override


tests = [
    {
        "name": "Example",
        "s": """
                7 6 4 2 1
                1 2 7 8 9
                9 7 6 2 1
                1 3 2 4 5
                8 6 4 4 1
                1 3 6 7 9
        """,
        "expected": 4,
    },
]


class ProblemDampener(Reports):
    @classmethod
    def safe(cls, report) -> bool:
        if super().safe(report):
            return True

        for i in range(len(report)):
            rr = report[:i] + report[i + 1 :]
            if super().safe(rr):
                return True

        return False


@test(tests=tests)
def partII_test(s: str) -> int:
    return ProblemDampener(s).count_safe_reports()


Test Example passed, for partII_test.
Success


<link href="style.css" rel="stylesheet"></link>


In [33]:
print(f"Part II: {ProblemDampener(puzzle).count_safe_reports()}")

Part II: 337


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>337</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
